# Setup

In [1]:
# Google Only
from google.colab import drive
drive.mount('/content/drive')
ROOT_FOLDER = '/content/drive/My Drive/Code/autocomplete_me/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
## Set Variables for Local and Cloud File Finding
import os
import sys
sys.path.append(ROOT_FOLDER)

In [3]:
!ls -l '/content/drive/My Drive/Code/autocomplete_me/src'

total 27
-rw------- 1 root root 2516 Jul 14 22:20 predict_utils.py
drwx------ 2 root root 4096 Jul 11 13:20 __pycache__
-rw------- 1 root root 3340 Jul 14 22:20 reader.py
-rw------- 1 root root 3580 Jul 14 22:20 train_model_baseline.py
-rw------- 1 root root 3203 Jul 14 22:20 train_utils.py
-rw------- 1 root root 9341 Jul 12 14:24 utils.py


In [4]:
from src import utils, reader, predict_utils, train_utils
from importlib import reload
reload(utils)
reload(reader)
reload(predict_utils)
reload(train_utils)

Using TensorFlow backend.


<module 'src.train_utils' from '/content/drive/My Drive/Code/autocomplete_me/src/train_utils.py'>

## Load Text Data

In [23]:
text = reader.read_bbc('entertainment')
content_type = 'BBC-Entertainment'

In [24]:
text[0]

'Blair buys copies of new Band Aid\n\nPrime Minister Tony Blair purchased two copies of the charity single Band Aid 20 in Edinburgh on Friday.\n\nStaff were surprised when the Prime Minister walked into HMV at 0900 GMT, accompanied by aides and local police. "When Mr Blair came in unannounced, we were all pretty gobsmacked," said HMV manager Clive Smith. "Our customer helper approached him... it was only then we realised he wanted to buy copies of the Band Aid single, rather than the latest Eminem album." Predicted chart-topper Do They Know it\'s Christmas? is expected to sell at least 300,000 copies by the time the new chart is announced on Sunday. However, the new version of the 1984 single is not going to be released in the US, despite being sold in many countries around the world. US record shops are stocking an import version of Do They Know It\'s Christmas, which is said to be selling very well in Los Angeles and New York. The original track was released in the US, and reached nu

# Modelling

## Process Text Data

In [25]:
sequences, num_words, word_idx, idx_word = train_utils.preprocess_text(text)

In [26]:
features, labels = train_utils.pass_sliding_window(sequences, sequence_len=10)

There are 125015 sequences.


In [27]:
labels = train_utils.one_hot_labels_and_improve_efficiency(labels)

Labels matrix shape:  (125015, 12541)
Labels matrix shape:  (125015, 12541)


In [28]:
# Create Test Train Set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.20, random_state=42, shuffle=True)

In [29]:
import gc
gc.enable()
del labels
gc.collect()

0

In [30]:
print('X_train shape: ', X_train.shape)
print('X_test shape: ', X_test.shape)
print('y_train shape: ', y_train.shape)
print('y_test shape: ', y_test.shape)

X_train shape:  (100012, 10)
X_test shape:  (25003, 10)
y_train shape:  (100012, 12541)
y_test shape:  (25003, 12541)


In [13]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

                       y_train:  1.4 GiB
                        y_test: 355.5 MiB
                      features: 12.6 MiB
                       X_train: 10.1 MiB
                        X_test:  2.5 MiB
                      word_idx: 576.1 KiB
                      idx_word: 576.1 KiB
                     sequences:  4.5 KiB
                          text:  4.2 KiB
                            __:  778.0 B


In [31]:
# Embedding Matrix
# embedding_matrix = utils.create_embedding_matrix(word_idx, num_words, '/Users/jaipancholi/data/glove.6B.100d.txt')
embedding_matrix = utils.create_embedding_matrix(word_idx, num_words, '/content/drive/My Drive/Code/autocomplete_me/data/glove.6B.100d.txt')
embedding_matrix

Glove Vectors loading with dimension 100
There were 1186 words without pre-trained embeddings.


/content/drive/My Drive/Code/autocomplete_me/src/utils.py:180: RuntimeWarning:

invalid value encountered in true_divide



array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.00656124, -0.04206555,  0.12508174, ..., -0.02506376,
         0.14220549,  0.04648907],
       [-0.01269503,  0.04080414,  0.00418698, ..., -0.12684801,
         0.15331151,  0.03447365],
       ...,
       [ 0.03130877,  0.01514814,  0.06958524, ...,  0.02720818,
         0.11345827,  0.06412452],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.05417296, -0.04813922,  0.13245429, ..., -0.20398643,
         0.01218687,  0.03430193]])

In [32]:
embedding_matrix.shape

(12541, 100)

# Design Model

In [15]:
from tensorflow.keras.models import load_model

In [16]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [17]:
def train_model(filepath, X_train, y_train, X_test, y_test, use_pretrained_model=False, model=False, epochs=100):
  if not model and not use_pretrained_model:
    print('Provide one of either model or use_pretrained_model.')
  elif model and use_pretrained_model:
      print('Provide one of either model or use_pretrained_model.')
  elif use_pretrained_model:
    model = load_model(model_filepath)
  
  callbacks = [
      EarlyStopping(monitor='val_accuracy', patience=25),
      ModelCheckpoint(f'{model_filepath}', save_best_only=True, save_weights_only=False, monitor='val_accuracy')
  ]

  history = model.fit(
      X_train, 
      y_train, 
      epochs=epochs, 
      batch_size=2048, 
      validation_data=(X_test, y_test), 
      verbose=1,
      callbacks=callbacks
  )

  return history

##V1

In [ ]:
model = Sequential()

model.add(    
    Embedding(
    input_dim=num_words,
    output_dim=embedding_matrix.shape[1],
    weights=[embedding_matrix],
    trainable=True)
)

model.add(LSTM(64))

model.add(Dropout(0.2))

model.add(Dense(128, activation='relu'))

# output layer
model.add(Dense(num_words, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_filepath = os.path.join(ROOT_FOLDER, 'models', f'{content_type}-custom-1.h5')
train_model(model_filepath, X_train, y_train, X_test, y_test, use_pretrained_model=False, model=model, epochs=500)

## V2

In [33]:
model = Sequential()

model.add(    
    Embedding(
    input_dim=num_words,
    output_dim=embedding_matrix.shape[1],
    weights=[embedding_matrix],
    trainable=True)
)

model.add(LSTM(256))

model.add(Dropout(0.2))

model.add(Dense(128, activation='relu'))

# output layer
model.add(Dense(num_words, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [34]:
model_filepath = os.path.join(ROOT_FOLDER, 'models', f'{content_type}-custom-2.h5')
train_model(model_filepath, X_train, y_train, X_test, y_test, use_pretrained_model=False, model=model, epochs=500)

Epoch 1/500
49/49 [==============================] - 6s 128ms/step - loss: 7.8605 - accuracy: 0.0609 - val_loss: 7.2590 - val_accuracy: 0.0643
Epoch 2/500
49/49 [==============================] - 6s 117ms/step - loss: 7.1513 - accuracy: 0.0653 - val_loss: 7.2325 - val_accuracy: 0.0643
Epoch 3/500
49/49 [==============================] - 6s 117ms/step - loss: 7.0817 - accuracy: 0.0653 - val_loss: 7.1817 - val_accuracy: 0.0643
Epoch 4/500
49/49 [==============================] - 6s 116ms/step - loss: 7.0136 - accuracy: 0.0653 - val_loss: 7.1765 - val_accuracy: 0.0643
Epoch 5/500
49/49 [==============================] - 6s 129ms/step - loss: 6.9652 - accuracy: 0.0653 - val_loss: 7.1617 - val_accuracy: 0.0643
Epoch 6/500
49/49 [==============================] - 6s 120ms/step - loss: 6.8910 - accuracy: 0.0687 - val_loss: 7.0955 - val_accuracy: 0.0721
Epoch 7/500
49/49 [==============================] - 6s 120ms/step - loss: 6.7755 - accuracy: 0.0746 - val_loss: 7.0222 - val_accuracy: 0.0777

## V3

In [37]:
model = Sequential()

model.add(    
    Embedding(
    input_dim=num_words,
    output_dim=embedding_matrix.shape[1],
    weights=[embedding_matrix],
    trainable=True)
)

model.add(LSTM(256))

model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))

# output layer
model.add(Dense(num_words, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [38]:
model_filepath = os.path.join(ROOT_FOLDER, 'models', f'{content_type}-custom-3.h5')
train_model(model_filepath, X_train, y_train, X_test, y_test, use_pretrained_model=False, model=model, epochs=500)

Epoch 1/500
49/49 [==============================] - 6s 126ms/step - loss: 7.8511 - accuracy: 0.0482 - val_loss: 7.2717 - val_accuracy: 0.0643
Epoch 2/500
49/49 [==============================] - 6s 116ms/step - loss: 7.1611 - accuracy: 0.0653 - val_loss: 7.2390 - val_accuracy: 0.0643
Epoch 3/500
49/49 [==============================] - 6s 118ms/step - loss: 7.0862 - accuracy: 0.0653 - val_loss: 7.1782 - val_accuracy: 0.0643
Epoch 4/500
49/49 [==============================] - 6s 129ms/step - loss: 7.0221 - accuracy: 0.0653 - val_loss: 7.1771 - val_accuracy: 0.0643
Epoch 5/500
49/49 [==============================] - 6s 118ms/step - loss: 6.9842 - accuracy: 0.0653 - val_loss: 7.1751 - val_accuracy: 0.0643
Epoch 6/500
49/49 [==============================] - 6s 129ms/step - loss: 6.9300 - accuracy: 0.0673 - val_loss: 7.1258 - val_accuracy: 0.0729
Epoch 7/500
49/49 [==============================] - 6s 119ms/step - loss: 6.8353 - accuracy: 0.0724 - val_loss: 7.0439 - val_accuracy: 0.0773

## V4

In [ ]:
model = Sequential()

model.add(    
    Embedding(
    input_dim=num_words,
    output_dim=embedding_matrix.shape[1],
    weights=[embedding_matrix],
    trainable=True)
)

model.add(LSTM(64, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))

model.add(Dropout(0.2))

model.add(LSTM(64))

model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))

# output layer
model.add(Dense(num_words, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_filepath = os.path.join(ROOT_FOLDER, 'models', f'{content_type}-custom-4.h5')
train_model(model_filepath, X_train, y_train, X_test, y_test, use_pretrained_model=False, model=model, epochs=500)

## V5

In [35]:
model = Sequential()

model.add(    
    Embedding(
    input_dim=num_words,
    output_dim=embedding_matrix.shape[1],
    weights=[embedding_matrix],
    trainable=True)
)

model.add(LSTM(256, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))

model.add(Dropout(0.2))

model.add(LSTM(256))

model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))

# output layer
model.add(Dense(num_words, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [36]:
model_filepath = os.path.join(ROOT_FOLDER, 'models', f'{content_type}-custom-5.h5')
train_model(model_filepath, X_train, y_train, X_test, y_test, use_pretrained_model=False, model=model, epochs=500)

Epoch 1/500
49/49 [==============================] - 9s 179ms/step - loss: 7.7985 - accuracy: 0.0581 - val_loss: 7.2744 - val_accuracy: 0.0643
Epoch 2/500
49/49 [==============================] - 8s 161ms/step - loss: 7.1743 - accuracy: 0.0653 - val_loss: 7.2723 - val_accuracy: 0.0643
Epoch 3/500
49/49 [==============================] - 8s 162ms/step - loss: 7.1487 - accuracy: 0.0653 - val_loss: 7.2874 - val_accuracy: 0.0643
Epoch 4/500
49/49 [==============================] - 8s 161ms/step - loss: 7.1412 - accuracy: 0.0653 - val_loss: 7.3000 - val_accuracy: 0.0643
Epoch 5/500
49/49 [==============================] - 8s 161ms/step - loss: 7.1355 - accuracy: 0.0653 - val_loss: 7.3265 - val_accuracy: 0.0643
Epoch 6/500
49/49 [==============================] - 8s 163ms/step - loss: 7.1272 - accuracy: 0.0653 - val_loss: 7.3156 - val_accuracy: 0.0643
Epoch 7/500
49/49 [==============================] - 8s 165ms/step - loss: 7.0507 - accuracy: 0.0653 - val_loss: 7.1872 - val_accuracy: 0.0643

## V6

In [ ]:
model = Sequential()

model.add(    
    Embedding(
    input_dim=num_words,
    output_dim=embedding_matrix.shape[1],
    weights=[embedding_matrix],
    trainable=True)
)

model.add(LSTM(256, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))

model.add(Dropout(0.5))

model.add(LSTM(256))

model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))

# output layer
model.add(Dense(num_words, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_filepath = os.path.join(ROOT_FOLDER, 'models', f'{content_type}-custom-6.h5')
train_model(model_filepath, X_train, y_train, X_test, y_test, use_pretrained_model=False, model=model, epochs=500)

In [ ]:
train_utils.plot_history(history)

# Comparison

### 100 Epochs

<tr>
    <th>LSTM Layers</th>
    <th>LSTM Cells per Layer</th>
    <th>Dropout %</th>
    <th>Validation Loss</th>
    <th>Validation Accuracy</th>
</tr>
<tr>
    <td>1</td>
    <td>64</td>
    <td>0.2</td>
    <td>8.9918</td>
    <td>0.2271</td>
</tr>
<tr>
    <td>1</td>
    <td>256</td>
    <td>0.2</td>
    <td>10.7950</td>
    <td>0.2354</td>
</tr>
<tr>
    <td>1</td>
    <td>256</td>
    <td>0.5</td>
    <td>8.9682</td>
    <td>0.2153</td>
</tr>
<tr>
    <td>2</td>
    <td>64</td>
    <td>0.2, 0.5</td>
    <td>6.9549</td>
    <td>0.1490</td>
</tr>
<tr>
    <td>2</td>
    <td>256</td>
    <td>0.2, 0.5</td>
    <td>7.4581</td>
    <td>0.1683</td>
</tr>
<tr>
    <td>2</td>
    <td>256</td>
    <td>0.5, 0.5</td>
    <td>7.3286</td>
    <td>0.1650</td>
</tr>

### 500 Epochs

<tr>
    <th>LSTM Layers</th>
    <th>LSTM Cells per Layer</th>
    <th>Dropout %</th>
    <th>Validation Loss</th>
    <th>Validation Accuracy</th>
</tr>
<tr>
    <td>1</td>
    <td>64</td>
    <td>0.2</td>
    <td>16.4351</td>
    <td>0.3229</td>
</tr>
<tr>
    <td>1</td>
    <td>256</td>
    <td>0.2</td>
    <td>22.5922</td>
    <td>0.3373</td>
</tr>
<tr>
    <td>1</td>
    <td>256</td>
    <td>0.5</td>
    <td>18.8511</td>
    <td>0.3329</td>
</tr>
<tr>
    <td>2</td>
    <td>64</td>
    <td>0.2, 0.5</td>
    <td>8.8127</td>
    <td>0.2230</td>
</tr>
<tr>
    <td>2</td>
    <td>256</td>
    <td>0.2, 0.5</td>
    <td>10.8240</td>
    <td>0.2854</td>
</tr>
<tr>
    <td>2</td>
    <td>256</td>
    <td>0.5, 0.5</td>
    <td>10.4388</td>
    <td>0.2666</td>
</tr>

# Generate Text Data

### Load Objects To Infer

In [ ]:
from tensorflow.keras.models import load_model
model_filepath = os.path.join(ROOT_FOLDER, 'models', f'{content_type}-custom-2.h5')
model = load_model(model_filepath)
TRAINING_LENGTH = 10

## Existing Sentences

In [ ]:
original_sequence, gen_list, a = predict_utils.generate_output(
    model,
    sequences,
    idx_word,
    seed_length=TRAINING_LENGTH,
    new_words=20,
    diversity=1,
    n_gen=1
)

/content/drive/My Drive/Code/autocomplete_me/src/predict_utils.py:42: RuntimeWarning:

divide by zero encountered in log



In [ ]:
' '.join(word for word in original_sequence)

'speeds the standard speed is 512kbps though faster connections are'

In [ ]:
' '.join(word for word in gen_list[0])

'< --- > available this year an analyst at a firm based broadcaster studio telecommunications company in response with 6 to 9 january'

In [ ]:
' '.join(word for word in a)

'< --- > available this breakthrough led to a dramatic increase in orders as we were suddenly able to satisfy the pent up'

## Custom Sentences

In [ ]:
sentence = 'Stocks of major large technology firms are becoming even more fragile even though'
predict_utils.generate_custom_sentence(sentence, word_idx, idx_word, model, new_words=20)

[None, 3, 546, 490, 45, 126, 13, 518, 150, 24, 9544, 150, 456]


ValueError: ignored

In [ ]:
sentence = 'However, there have been many instances of'
predict_utils.generate_custom_sentence(sentence, word_idx, idx_word, model, new_words=50)

[None, 56, 18, 46, 67, 7424, 3]


ValueError: ignored